In [7]:
import os
import re
import pandas as pd
import dateparser
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# Paths — change these to your own directories
# -----------------------------------------------------------------------------
stock_dir     = '/Users/User/Desktop/DIA/Data/Raw Data/stock_data'
sentiment_dir = '/Users/User/Desktop/DIA/Data/Raw Data/news_sentiment'
forecast_dir  = '/Users/User/Desktop/DIA/Test/Forecast by XGBoost (Rolling window with regularization)'

updated_stock = '/Users/User/Desktop/DIA/Test/stock_data'
graph_dir     = '/Users/User/Desktop/DIA/Test/graph'
merged_dir    = '/Users/User/Desktop/DIA/Test/Merged_Data_For_RL'
train_dir     = os.path.join(merged_dir, 'train')
test_dir      = os.path.join(merged_dir, 'test')

# create folders
for d in (updated_stock, graph_dir, merged_dir, train_dir, test_dir):
    os.makedirs(d, exist_ok=True)

In [8]:
# -----------------------------------------------------------------------------
# Utility functions
# -----------------------------------------------------------------------------
def compute_rsi(series, period=14):
    delta    = series.diff()
    gain     = delta.clip(lower=0)
    loss     = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()
    rs       = avg_gain / avg_loss
    rsi      = 100 - (100 / (1 + rs))
    rsi.iloc[0] = 50
    return rsi

def extract_date(text):
    if pd.isna(text):
        return None
    txt = re.sub(r'\bSept\.?\b','Sep', text)
    for line in txt.splitlines()[::-1]:
        dt = dateparser.parse(line.strip())
        if dt:
            return dt.date()
    return None

In [9]:
# -----------------------------------------------------------------------------
# 1) Compute technical indicators & save updated CSV + graphs
# -----------------------------------------------------------------------------
for fname in os.listdir(stock_dir):
    if not fname.lower().endswith('.csv'):
        continue

    path = os.path.join(stock_dir, fname)
    df   = pd.read_csv(path, parse_dates=['Date']).sort_values('Date').reset_index(drop=True)

    # moving averages & RSI
    df['MA5']  = df['Close'].rolling(5,  min_periods=1).mean().round(2)
    df['MA20'] = df['Close'].rolling(20, min_periods=1).mean().round(2)
    df['RSI']  = compute_rsi(df['Close'], period=14).round(2)

    # MACD & signal line
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD']        = (ema12 - ema26).round(2)
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean().round(2)

    # save updated CSV
    out_csv = os.path.join(updated_stock, fname)
    df.to_csv(out_csv, index=False)
    print(f"[1] Indicators added → {fname}")

    # plot and save
    fig, (ax1, ax2) = plt.subplots(2,1, figsize=(12,8), sharex=True)
    ax1.plot(df['Date'], df['Close'],    label='Close',  linewidth=2)
    ax1.plot(df['Date'], df['MA5'],      '--', label='MA5')
    ax1.plot(df['Date'], df['MA20'],     '--', label='MA20')
    ax1.set_ylabel('Price')
    ax1.legend(loc='upper left')
    ax1.set_title(fname.replace('.csv',''))

    ax2.plot(df['Date'], df['MACD'],        label='MACD')
    ax2.plot(df['Date'], df['Signal_Line'], label='Signal Line')
    ax2.set_ylabel('MACD')
    ax2.set_xlabel('Date')
    ax2.legend(loc='upper left')

    plt.tight_layout()
    out_png = os.path.join(graph_dir, fname.replace('.csv','.png'))
    plt.savefig(out_png, dpi=150)
    plt.close()
    print(f"     → Graph saved as {out_png}")


[1] Indicators added → MSFT_stock_data_2022-01-01_to_2024-12-31.csv
     → Graph saved as /Users/User/Desktop/DIA/Test/graph/MSFT_stock_data_2022-01-01_to_2024-12-31.png
[1] Indicators added → VZ_stock_data_2022-01-01_to_2024-12-31.csv
     → Graph saved as /Users/User/Desktop/DIA/Test/graph/VZ_stock_data_2022-01-01_to_2024-12-31.png
[1] Indicators added → CSCO_stock_data_2022-01-01_to_2024-12-31.csv
     → Graph saved as /Users/User/Desktop/DIA/Test/graph/CSCO_stock_data_2022-01-01_to_2024-12-31.png
[1] Indicators added → KO_stock_data_2022-01-01_to_2024-12-31.csv
     → Graph saved as /Users/User/Desktop/DIA/Test/graph/KO_stock_data_2022-01-01_to_2024-12-31.png
[1] Indicators added → HD_stock_data_2022-01-01_to_2024-12-31.csv
     → Graph saved as /Users/User/Desktop/DIA/Test/graph/HD_stock_data_2022-01-01_to_2024-12-31.png
[1] Indicators added → V_stock_data_2022-01-01_to_2024-12-31.csv
     → Graph saved as /Users/User/Desktop/DIA/Test/graph/V_stock_data_2022-01-01_to_2024-12-31.pn

In [10]:
# -----------------------------------------------------------------------------
# 2) Merge updated stock with sentiment → merged_dir (no forecast yet)
# -----------------------------------------------------------------------------
for fname in os.listdir(updated_stock):
    if not fname.lower().endswith('.csv'):
        continue

    stock_df = pd.read_csv(os.path.join(updated_stock, fname), parse_dates=['Date'])
    stock_df.sort_values('Date', inplace=True)

    symbol   = fname.split('_')[0]
    sent_path = os.path.join(sentiment_dir, f"{symbol}_news_sentiment.csv")
    if not os.path.isfile(sent_path):
        print(f"[2] ⚠ no sentiment for {symbol}, skipping")
        continue

    sent_df = pd.read_csv(sent_path)
    sent_df['headline'] = sent_df['headline']\
        .astype(str)\
        .str.replace(r'\bSept\.?\b','Sep', regex=True)
    sent_df['Date']     = sent_df['headline'].apply(extract_date)
    sent_df.dropna(subset=['Date'], inplace=True)
    sent_df['Date']     = pd.to_datetime(sent_df['Date'])

    sent_agg = sent_df.groupby('Date', as_index=False)['sentiment'].mean()

    merged = stock_df.merge(sent_agg, on='Date', how='left')
    merged['sentiment'] = merged['sentiment'].fillna(0.0)
    merged.rename(columns={'sentiment':'Sentiment_Score'}, inplace=True)

    # round numeric columns
    num_cols = merged.select_dtypes(include=['float64','int64']).columns
    merged[num_cols] = merged[num_cols].round(2)

    out_path = os.path.join(merged_dir, fname)
    merged.to_csv(out_path, index=False)
    print(f"[2] Stock+Sentiment → {fname}")

[2] Stock+Sentiment → MSFT_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → VZ_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → CSCO_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → KO_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → HD_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → V_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → MMM_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → PFE_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → NKE_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → CAT_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → GS_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → JNJ_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → DD_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → TRV_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Sentiment → JPM_stock_data_2022-01-01_to_2024-12-31.csv
[2] Stock+Senti

In [11]:
#  -----------------------------------------------------------------------------
# 3) Split merged_dir into train (pre-2024) and test (2024+)
# -----------------------------------------------------------------------------
for fname in os.listdir(merged_dir):
    if not fname.lower().endswith('.csv'):
        continue

    full_in = os.path.join(merged_dir, fname)
    df      = pd.read_csv(full_in, parse_dates=['Date'])

    train_df = df[df['Date'] < '2024-01-01']
    test_df  = df[df['Date'] >= '2024-01-01']

    if not train_df.empty:
        train_df.to_csv(os.path.join(train_dir, fname), index=False)
    else:
        print(f"⚠️  {fname} has no train rows")

    if not test_df.empty:
        test_df.to_csv(os.path.join(test_dir, fname), index=False)
    else:
        print(f"⚠️  {fname} has no test rows")


In [12]:
# -----------------------------------------------------------------------------
# 3) Split merged_dir into train (pre-2024) and test (2024+)
# -----------------------------------------------------------------------------
for fname in os.listdir(merged_dir):
    if not fname.lower().endswith('.csv'):
        continue

    full_in = os.path.join(merged_dir, fname)
    df      = pd.read_csv(full_in, parse_dates=['Date'])

    train_df = df[df['Date'] < '2024-01-01']
    test_df  = df[df['Date'] >= '2024-01-01']

    if not train_df.empty:
        train_df.to_csv(os.path.join(train_dir, fname), index=False)
    else:
        print(f"⚠️  {fname} has no train rows")

    if not test_df.empty:
        test_df.to_csv(os.path.join(test_dir, fname), index=False)
    else:
        print(f"⚠️  {fname} has no test rows")

# -----------------------------------------------------------------------------
# 4) Merge forecast into test_dir files only, rename & round
# -----------------------------------------------------------------------------
# build forecast map
forecast_map = {}
for f in os.listdir(forecast_dir):
    if f.endswith('_close_sentiment.csv'):
        symbol = f.split('_')[0]
        forecast_map[symbol] = os.path.join(forecast_dir, f)
print("Available forecast symbols:", list(forecast_map.keys()))

for fname in os.listdir(test_dir):
    if not fname.lower().endswith('.csv'):
        continue

    path_in = os.path.join(test_dir, fname)
    df      = pd.read_csv(path_in, parse_dates=['Date'])
    df['Date'] = df['Date'].dt.normalize()

    symbol = fname.split('_')[0]
    print(f"[4] looking for forecast for {symbol}")

    if symbol not in forecast_map:
        print(f"[4] ⚠ no forecast for {symbol}")
        continue

    fdf = pd.read_csv(forecast_map[symbol], parse_dates=['Date'])
    fdf['Date'] = fdf['Date'].dt.normalize()

    # merge and rename
    df = df.merge(
        fdf[['Date','Predicted_Close']],
        on='Date', how='left'
    ).rename(columns={'Predicted_Close':'Predicted_Next_Close'})

    # drop rows without a forecast
    df.dropna(subset=['Predicted_Next_Close'], inplace=True)

    # round the two columns
    df['Predicted_Next_Close'] = df['Predicted_Next_Close'].round(2)
    df['Sentiment_Score']      = df['Sentiment_Score'].round(2)

    df.to_csv(path_in, index=False)
    print(f"[4] Forecast merged → {fname}")

Available forecast symbols: ['HD', 'MMM', 'INTC', 'DD', 'MCD', 'XOM', 'PG', 'VZ', 'CAT', 'CSCO', 'WBA', 'IBM', 'NKE', 'CVX', 'AAPL', 'KO', 'V', 'PFE', 'TRV', 'GS', 'JNJ', 'AXP', 'JPM', 'DIS', 'BA', 'MRK', 'UNH', 'WMT', 'MSFT', 'RTX']
[4] looking for forecast for MSFT
[4] Forecast merged → MSFT_stock_data_2022-01-01_to_2024-12-31.csv
[4] looking for forecast for VZ
[4] Forecast merged → VZ_stock_data_2022-01-01_to_2024-12-31.csv
[4] looking for forecast for CSCO
[4] Forecast merged → CSCO_stock_data_2022-01-01_to_2024-12-31.csv
[4] looking for forecast for KO
[4] Forecast merged → KO_stock_data_2022-01-01_to_2024-12-31.csv
[4] looking for forecast for HD
[4] Forecast merged → HD_stock_data_2022-01-01_to_2024-12-31.csv
[4] looking for forecast for V
[4] Forecast merged → V_stock_data_2022-01-01_to_2024-12-31.csv
[4] looking for forecast for MMM
[4] Forecast merged → MMM_stock_data_2022-01-01_to_2024-12-31.csv
[4] looking for forecast for PFE
[4] Forecast merged → PFE_stock_data_2022-01-0